# DICOM Histograms

## Global variables & Immports

In [ ]:
# Path to DICOM directory
DICOM_PATH = "/home/pyuser/data/Paradise_DICOMs"

# Path to Histogram directory
HISTOGRAM_PATH = "/home/pyuser/data/Paradise_Histograms"

import os
from tqdm import tqdm
import pydicom
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import warnings

# Set seaborn style for better looking plots
sns.set_style("whitegrid")
sns.set_palette("husl")

# Suppress the specific pydicom character encoding warning
warnings.filterwarnings("ignore", category=UserWarning, module="pydicom.charset")

## Graphs creation

### Bits Used Distribution

In [ ]:
def analyze_bits_used_distribution():
    """
    Analyze the distribution of bits used across all DICOM files
    and create a summary histogram.
    """
    # Get all DICOM files
    dicom_files = []
    for root, dirs, files in os.walk(DICOM_PATH):
        for file in files:
            if file.lower().endswith(('.dcm', '.dicom')):
                dicom_files.append(os.path.join(root, file))
    
    print(f"Analyzing bits used distribution for {len(dicom_files)} DICOM files...")
    
    bits_used_list = []
    bits_allocated_list = []
    file_info = []
    
    for dicom_file in tqdm(dicom_files, desc="Analyzing files"):
        try:
            # Read DICOM file
            ds = pydicom.dcmread(dicom_file)
            
            # Get pixel data
            pixel_array = ds.pixel_array
            
            # Get max pixel value
            max_pixel = np.max(pixel_array)
            
            # Get bits allocated
            bits_allocated = getattr(ds, 'BitsAllocated', 16)
            
            # Calculate bits used
            if max_pixel > 0:
                bits_used = int(np.ceil(np.log2(max_pixel + 1)))
            else:
                bits_used = 1
            
            bits_used_list.append(bits_used)
            bits_allocated_list.append(bits_allocated)
            file_info.append({
                'file': os.path.basename(dicom_file),
                'bits_used': bits_used,
                'bits_allocated': bits_allocated,
                'max_pixel': max_pixel
            })
            
        except Exception as e:
            print(f"Error processing {dicom_file}: {str(e)}")
            continue
    
    # Create distribution plot
    plt.figure(figsize=(12, 8))
    
    # Create subplot for bits used distribution
    plt.subplot(2, 1, 1)
    sns.histplot(data=bits_used_list, bins=range(min(bits_used_list), max(bits_used_list) + 2), 
                 color='steelblue', alpha=0.7, edgecolor='black')
    plt.title('Distribution of Bits Used Across All DICOM Files', fontsize=14, fontweight='bold')
    plt.xlabel('Bits Used', fontsize=12)
    plt.ylabel('Number of Files', fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Add statistics
    unique_bits_used = sorted(list(set(bits_used_list)))
    stats_text = f'Unique bits used values: {unique_bits_used}\nTotal files: {len(bits_used_list)}\nMost common: {max(set(bits_used_list), key=bits_used_list.count)} bits'
    plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8), fontsize=10)
    
    # Create subplot for bits allocated vs bits used comparison
    plt.subplot(2, 1, 2)
    x_pos = np.arange(len(unique_bits_used))
    counts_used = [bits_used_list.count(bit) for bit in unique_bits_used]
    
    bars = plt.bar(x_pos, counts_used, alpha=0.7, edgecolor='black')
    
    # Color bars based on bits used (same color scheme as histograms)
    for i, (bar, bits) in enumerate(zip(bars, unique_bits_used)):
        if bits <= 8:
            bar.set_color('lightblue')
        elif bits <= 10:
            bar.set_color('blue')
        elif bits <= 12:
            bar.set_color('green')
        elif bits <= 13:
            bar.set_color('orange')
        elif bits <= 14:
            bar.set_color('red')
        elif bits <= 15:
            bar.set_color('purple')
        else:
            bar.set_color('darkred')
    
    plt.title('File Count by Bits Used (Colored by Histogram Color Scheme)', fontsize=14, fontweight='bold')
    plt.xlabel('Bits Used', fontsize=12)
    plt.ylabel('Number of Files', fontsize=12)
    plt.xticks(x_pos, unique_bits_used)
    plt.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for i, v in enumerate(counts_used):
        plt.text(i, v + max(counts_used) * 0.01, str(v), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    
    # Save the distribution plot
    output_path = os.path.join(HISTOGRAM_PATH, "bits_used_distribution.png")
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print detailed statistics
    print(f"\n=== BITS USED DISTRIBUTION ANALYSIS ===")
    print(f"Total files processed: {len(bits_used_list)}")
    print(f"Bits used range: {min(bits_used_list)} - {max(bits_used_list)}")
    print(f"Most common bits used: {max(set(bits_used_list), key=bits_used_list.count)} ({bits_used_list.count(max(set(bits_used_list), key=bits_used_list.count))} files)")
    print(f"\nBreakdown by bits used:")
    for bits in unique_bits_used:
        count = bits_used_list.count(bits)
        percentage = (count / len(bits_used_list)) * 100
        print(f"  {bits} bits: {count} files ({percentage:.1f}%)")
    
    return file_info, bits_used_list

# Execute the analysis
file_info, bits_distribution = analyze_bits_used_distribution()


### Histogram per DICOM file

In [ ]:
def create_dicom_histograms():
    """
    Create histograms for all DICOM files in the DICOM_PATH directory
    and save them to the HISTOGRAM_PATH directory.
    """
    # Create histogram directory if it doesn't exist
    Path(HISTOGRAM_PATH).mkdir(parents=True, exist_ok=True)
    
    # Get all DICOM files
    dicom_files = []
    for root, dirs, files in os.walk(DICOM_PATH):
        for file in files:
            if file.lower().endswith(('.dcm', '.dicom')):
                dicom_files.append(os.path.join(root, file))
    
    print(f"Found {len(dicom_files)} DICOM files")
    
    for dicom_file in tqdm(dicom_files, desc="Creating histograms"):
        try:
            # Read DICOM file
            ds = pydicom.dcmread(dicom_file)
            
            # Get pixel data
            pixel_array = ds.pixel_array
            
            # Get pixel value statistics
            min_pixel = np.min(pixel_array)
            max_pixel = np.max(pixel_array)
            
            # Get bits allocated from DICOM header
            bits_allocated = getattr(ds, 'BitsAllocated', None)
            if bits_allocated is None:
                # Fallback: estimate from pixel data type
                if pixel_array.dtype == np.uint8:
                    bits_allocated = 8
                elif pixel_array.dtype == np.uint16:
                    bits_allocated = 16
                else:
                    bits_allocated = 16  # Default fallback
            
            # Get photometric interpretation (MONOCHROME1 or MONOCHROME2)
            photometric = getattr(ds, 'PhotometricInterpretation', 'Unknown')
            if 'MONOCHROME' in photometric:
                if '1' in photometric:
                    monochrome_type = 'MONOCHROME1'
                elif '2' in photometric:
                    monochrome_type = 'MONOCHROME2'
                else:
                    monochrome_type = photometric
            else:
                monochrome_type = photometric
            
            # Calculate actual bits used (based on maximum pixel value)
            if max_pixel > 0:
                bits_used = int(np.ceil(np.log2(max_pixel + 1)))
            else:
                bits_used = 1
            
            # Determine color based on bits used (more informative than bits allocated)
            if bits_used <= 8:
                color = 'lightblue'
            elif bits_used <= 10:
                color = 'blue'
            elif bits_used <= 12:
                color = 'green'
            elif bits_used <= 13:
                color = 'orange'
            elif bits_used <= 14:
                color = 'red'
            elif bits_used <= 15:
                color = 'purple'
            else:
                color = 'darkred'
            
            # Determine bins based on bits allocated (for proper histogram range)
            if bits_allocated <= 8:
                bins = min(256, max_pixel - min_pixel + 1)
            elif bits_allocated <= 12:
                bins = min(4096, max_pixel - min_pixel + 1)
            elif bits_allocated <= 14:
                bins = min(16384, max_pixel - min_pixel + 1)
            else:
                bins = min(65536, max_pixel - min_pixel + 1)
            
            # Ensure we have at least 50 bins for good visualization
            bins = max(bins, 50)
            
            # Create histogram using matplotlib with better color control
            plt.figure(figsize=(10, 6))
            n, bins_edges, patches = plt.hist(pixel_array.flatten(), bins=bins, alpha=0.7, 
                                            edgecolor='black', linewidth=0.5)
            
            # Apply color to all histogram bars
            for patch in patches:
                patch.set_facecolor(color)
            
            plt.title(f'Pixel Value Distribution - {os.path.basename(dicom_file)}', 
                     fontsize=14, fontweight='bold')
            plt.xlabel('Pixel Value', fontsize=12)
            plt.ylabel('Frequency', fontsize=12)
            
            # Seaborn already provides a nice grid, but we can customize it
            plt.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
            
            # Add comprehensive information text box with proper newlines
            info_text = f'Bits Allocated: {bits_allocated}\nBits Used: {bits_used}\n{monochrome_type}\nMin Pixel Value: {min_pixel}\nMax Pixel Value: {max_pixel}'
            plt.text(0.02, 0.98, info_text, 
                    transform=plt.gca().transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                    fontsize=10)
            
            # Save histogram
            base_name = os.path.splitext(os.path.basename(dicom_file))[0]
            output_path = os.path.join(HISTOGRAM_PATH, f"{base_name}_histogram.png")
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            plt.close()
            
        except Exception as e:
            print(f"Error processing {dicom_file}: {str(e)}")
            continue

# Execute the function
create_dicom_histograms()


### Global histogram